In [1]:
import pandas as pd
from services.faiss import FaissRAG

d:\askello\askello-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../data/CRM & LMS task.csv")

answers = df['Answers '].tolist()

In [3]:
faiss_rag = FaissRAG(answers)

:FAISS IVF index created successfully


In [4]:
query = "What is the settlement period!"

In [5]:
faiss_rag.search(query)

Time required for FAISS Search: 0.02300572395324707


['The settlement period is T+1, meaning the transaction is settled on the next business day after the trade date.',
 'Settlement happens on a T+1 basis, excluding holidays. If you’ve not received funds beyond this, please raise a ticket with trade details.',
 'T+1 means the settlement of stocks occurs one business day after the trade date.']